# Weekly Investment Summary
#### Done
* Use configuration file for confidential data (E-Mail, Google Sheet, etc.) **DONE**
* Write for-loop in E-Mail body for measures of single cryptocurrencies **DONE**
* Format decimals in pie-chart **DONE**
* Make currency and measure (Close, Max, Min, Open) flexible **DONE**
* Sort list 'Asset' alphabeticially **DONE**

#### To Do
* Write README and define format of Google Sheet input, G-Mail set-up & file paths
* Write for-loop for E-Mail attachment
* Make start date flexible
* Replace text with a table showing details by type

In [62]:
#Setting up my environment
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import datetime as dt

import plotly.express as px
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf

import gspread as gs

import kaleido

import smtplib
from pathlib import Path
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate
from email import encoders

import configparser

import tempfile

In [63]:
#Configuration File
config = configparser.ConfigParser()

config.read('Weekly_Investment_Summary.ini')
config.sections()
config['Google_Sheet']['API_filename']

'google_API.json'

In [64]:
#DEFINITIONS
#Google Sheet
API_filename = config['Google_Sheet']['API_filename']
gs_url = config['Google_Sheet']['gs_url']
worksheet = config['Google_Sheet']['worksheet']

#Historical Data
currency = config['Price_Data']['currency']
metric = config['Price_Data']['metric']

#Plot/Attachment paths
total_holdings_image_path = tempfile.NamedTemporaryFile(suffix = '.png').name
holdings_by_type_image_path = tempfile.NamedTemporaryFile(suffix = '.png').name
allocation_of_holdings_image_path = tempfile.NamedTemporaryFile(suffix = '.png').name

#E-Mail
send_from = 'DLA'
send_to = config['E_Mail']['send_to']
username = config['E_Mail']['username']
password = config['E_Mail']['password']

In [65]:
#DATA IMPORT
#GOOGLE SHEET TRANSACTIONS
gs1 = gs.service_account(API_filename)
gs2 = gs1.open_by_url(gs_url)
gs3 = gs2.worksheet(worksheet)
gs4 = gs3.get_all_records()

trans_data = pd.DataFrame(gs4)

#Formatting of date column
trans_data['Date'] = pd.to_datetime(trans_data['Date'])
trans_data = trans_data[['Date','Asset','Quantity']]

#DAILY CLOSE PRICES
#Set parameters
start = dt.datetime(2020,1,1)
end = dt.datetime.now()
crypto = sorted(trans_data['Asset'].unique().tolist())
colnames = []
first = True

#Import historical data
for ticker in crypto:
    try:
        data = web.DataReader(f'{ticker}', 'yahoo', start, end)
        if first:
            combined = data[[metric]].copy()
            colnames.append(ticker)
            combined.columns = colnames
            first = False
        else:
            combined = combined.join(data[metric])
            colnames.append(ticker)
            combined.columns = colnames
    except:
        print (f'could not load Data for {ticker}')

#Re-shape data frame
combined.reset_index(inplace=True)
hist_data = pd.melt(combined, id_vars='Date', var_name='Asset',value_name=metric)

/tmp/ipykernel_4052/2531231751.py:11: UserWarning:

Parsing '13.03.2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/tmp/ipykernel_4052/2531231751.py:11: UserWarning:

Parsing '30.11.2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/tmp/ipykernel_4052/2531231751.py:11: UserWarning:

Parsing '15.12.2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.



could not load Data for 


In [66]:
#DATA MANIPULATION
#JOIN DATA
comb_data = pd.merge(trans_data,hist_data,how='right',on=['Date','Asset'])

#Calculation of accumulative, continuous sum for each crypto type, merge into one column and calculate the continuous value
for ticker in crypto:
    comb_data[f'Acc_Quantity_{ticker}'] = comb_data[(comb_data['Asset'] == f'{ticker}')]['Quantity'].cumsum().ffill()
    comb_data[f'Acc_Quantity_{ticker}'] = comb_data[f'Acc_Quantity_{ticker}'].fillna(value=0)

comb_data['Acc_Quantity'] = 0

for ticker in crypto:
    comb_data['Acc_Quantity'] += comb_data[f'Acc_Quantity_{ticker}']

comb_data['Pur_Value'] = comb_data['Quantity'] * comb_data[metric]
comb_data['Pur_Value'].fillna(value=0,inplace=True)

for ticker in crypto:
    comb_data[f'Acc_Pur_Value_{ticker}'] = comb_data[(comb_data['Asset'] == f'{ticker}')]['Pur_Value'].cumsum().ffill()
    comb_data[f'Acc_Pur_Value_{ticker}'] = comb_data[f'Acc_Pur_Value_{ticker}'].fillna(value=0)

comb_data['Acc_Pur_Value'] = 0

for ticker in crypto:
    comb_data['Acc_Pur_Value'] += comb_data[f'Acc_Pur_Value_{ticker}']

comb_data['Value'] = comb_data['Acc_Quantity'] * comb_data[metric]

comb_data['Pur_Value'] = comb_data['Quantity'] * comb_data[metric]
comb_data['Pur_Value'].fillna(value=0,inplace=True)

comb_data = comb_data[['Date','Asset','Quantity',metric,'Acc_Quantity','Value','Pur_Value','Acc_Pur_Value']]

#Total holdings
total_holdings = comb_data[['Date','Value','Pur_Value']].groupby('Date').sum()
total_holdings['Pur_Value'] = total_holdings['Pur_Value'].cumsum().ffill()

In [67]:
#DATA ANALYSIS
#Total holdings measures
current_value = total_holdings.sort_values('Date',ascending=False).head(8)['Value']
max_value = round(total_holdings['Value'].max(), 2)
max_value_date = total_holdings['Value'].idxmax().strftime('%B %#d, %Y')
purchase_value = total_holdings.sort_values('Date',ascending=False).head(1)['Pur_Value']

purchase_value_only = round(purchase_value[0],2)
current_value_only = round(current_value[0],2)
past_value_only = round(current_value[1],2)
prior_week_value_only = round(current_value[7],2)

change_prior_day = round((current_value_only/past_value_only*100)-100,2)
change_prior_week = round((current_value_only/prior_week_value_only*100)-100,2)
change_purchase = round((current_value_only/purchase_value_only*100)-100,2)

total_performance = round((current_value_only-purchase_value_only),2)

#Measures by cryptocurrency
#All-time high
max_asset_value = dict.fromkeys(crypto, 0)
#All-time high date
max_asset_date = dict.fromkeys(crypto, 0)
#Current
current_asset_value = dict.fromkeys(crypto, 0)
#Prior week
prior_week_asset_value = dict.fromkeys(crypto, 0)
#Current purchase value
current_purchase_value = dict.fromkeys(crypto, 0)
for ticker in crypto:
    asset_data = comb_data[(comb_data['Asset'] == f'{ticker}')]
    if not asset_data.empty:
        asset_value = max(asset_data['Value'], default=0)
        max_asset_value[ticker] = asset_value
        asset_value = asset_data.set_index('Date')['Value'].idxmax()
        max_asset_date[ticker] = asset_value.strftime('%B %#d, %Y')
        asset_value = asset_data.sort_values('Date', ascending=False).head(1)['Value']
        current_asset_value[ticker] = asset_value.iloc[0]
        asset_value = asset_data.sort_values('Date', ascending=False).head(8)['Value']
        prior_week_asset_value[ticker] = asset_value.iloc[7]
        asset_value = asset_data.sort_values('Date', ascending=False).head(1)['Acc_Pur_Value']
        current_purchase_value[ticker] = asset_value.iloc[0]

In [68]:
#DATA VISUALIZATION
#Development total holdings
fig = px.line(total_holdings,x=total_holdings.index,y=['Value','Pur_Value'],width=2000,height=1000)
fig.update_layout(title_text='Development Total Holdings', title_x=0.5)
fig.update_yaxes(title=f'Value in {currency}')
fig.write_image(total_holdings_image_path)
fig.show()

#Development by Type
fig_2 = px.line(comb_data,x='Date',y='Value', color='Asset',width=2000,height=1000)
fig_2.update_layout(title_text='Development Holdings by Type', title_x=0.5)
fig_2.update_yaxes(title=f'Value in {currency}')
fig_2.write_image(holdings_by_type_image_path)

#Allocation of Holdings
distinct_values = comb_data['Asset'].nunique()

fig_3 = px.pie(round(comb_data.sort_values('Date', ascending=False).head(distinct_values),2), values='Value', names='Asset', width=1200, height=800, hole=.5)
fig_3.update_traces(textposition='outside',
                    texttemplate='<b>%{label} </b><br>%{value} {currency} <br>%{percent:.2%}')
fig_3.add_annotation(text=f'<b>{current_value_only} {currency}',
                    font_size=30,
                    showarrow=False)
fig_3.update_layout(title_text='Allocation of Holdings', title_x=0.5, showlegend=False)
fig_3.write_image(allocation_of_holdings_image_path)

In [69]:
#E-MAIL NOTIFICATION
#Shows information about total holdings in text format
subject = 'Weekly Investment Report'
body =  f'Current total holdings: {current_value_only}{currency} ({change_prior_week}% to prior week)\n\
   - Gain/Loss of {total_performance}{currency} ({change_purchase}%) to purchase value, which was {purchase_value_only}{currency}\n\
   - All-time high of {max_value}{currency} on {max_value_date}\n\n'

msg = MIMEMultipart()

msg['From'] = send_from
msg['To'] = send_to
msg['Date'] = formatdate(localtime=True)
msg['Subject'] = subject
server = "smtp.gmail.com"
port = 587
use_tls = True

msg.attach(MIMEText(body))

#Calculates details and shows information in text format
for ticker in crypto:
    latest = round(current_asset_value[ticker],2)
    if latest == 0:
        continue
    prior = round(prior_week_asset_value[ticker],2)
    change_prior_week = round((latest/prior*100)-100,2)
    purchase = round(current_purchase_value[ticker],2)
    performance = round((latest - purchase),2)
    change_to_purchase = round((latest/purchase*100)-100,2)
    all_time_high = round(max_asset_value[ticker],2)
    date = max_asset_date[ticker]

    body_2 = f'     t/o {ticker}: {latest}{currency} ({change_prior_week}% to prior week)\n\
       - Gain/Loss of {performance}{currency} ({change_to_purchase}%) to purchase value, which was {purchase}{currency}\n\
       - All-time high of {all_time_high}{currency} on {date}\n\n'

    msg.attach(MIMEText(body_2))

#For-loop does not work for attachment?!?
part = MIMEBase('application', "octet-stream")
with open(total_holdings_image_path, 'rb') as file:
    part.set_payload(file.read())
encoders.encode_base64(part)
part.add_header('Content-Disposition',
                'attachment; filename="Total_Holdings.png"')
msg.attach(part)

part = MIMEBase('application', "octet-stream")
with open(holdings_by_type_image_path, 'rb') as file:
    part.set_payload(file.read())
encoders.encode_base64(part)
part.add_header('Content-Disposition',
                'attachment; filename="Holdings_by_Type.png"')
msg.attach(part)

part = MIMEBase('application', "octet-stream")
with open(allocation_of_holdings_image_path, 'rb') as file:
    part.set_payload(file.read())
encoders.encode_base64(part)
part.add_header('Content-Disposition',
                'attachment; filename="Allocation_of_Holdings.png"')
msg.attach(part)

smtp = smtplib.SMTP(server, port)
if use_tls:
    smtp.starttls()
smtp.login(username, password)
smtp.sendmail(send_from, send_to, msg.as_string())
smtp.quit()

print ("Message Sent")

Message Sent


In [70]:
# TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST
#Overview Table
# overview_table = pd.DataFrame({'Current Value':pd.Series(current_asset_value),
#                                 'Purchase Value':pd.Series(current_purchase_value),
#                                 'All-time High':pd.Series(max_asset_value),
#                                 'All-time High Date':pd.Series(max_asset_date),
#                                 'Prior Week Value':pd.Series(prior_week_asset_value)})

# overview_table['Gain/Loss'] = overview_table['Current Value'] - overview_table['Purchase Value']
# overview_table['Gain/Loss %'] = (overview_table['Current Value'] / overview_table['Purchase Value']*100)-100
# overview_table['Prior Week %'] = (overview_table['Current Value'] / overview_table['Prior Week Value']*100)-100

# overview_table = overview_table[['Current Value','Purchase Value','Gain/Loss','Gain/Loss %','Prior Week Value','Prior Week %','All-time High','All-time High Date']]

# overview_table.round(2)

In [71]:
#E-MAIL NOTIFICATION
#Define message
# subject = 'Weekly Crypto Report'
# body =  f'Current total holdings: {current_value_only}€ ({change_prior_week}% to prior week)\n\
#    - Gain/Loss of {total_performance}€ ({change_purchase}%) to purchase value, which was {purchase_value_only}€\n\
#    - All-time high of {max_value}€ on {max_value_date}\n\n'

# msg = MIMEMultipart()

# msg['From'] = send_from
# msg['To'] = send_to
# msg['Date'] = formatdate(localtime=True)
# msg['Subject'] = subject
# server = "smtp.gmail.com"
# port = 587
# use_tls = True

# html = """\
#         <html>
#           <head></head>
#           <body>
#             {0}
#           </body>
#         </html>
# """.format(overview_table.to_html(index=False))

# part1 = MIMEText(html, 'html')
# msg.attach(part1)

# msg.attach(MIMEText(body))

# for ticker in crypto:
#     latest = round(current_asset_value[ticker],2)
#     prior = round(prior_week_asset_value[ticker],2)
#     change_prior_week = round((latest/prior*100)-100,2)
#     purchase = round(current_purchase_value[ticker],2)
#     performance = round((latest - purchase),2)
#     change_to_purchase = round((latest/purchase*100)-100,2)
#     all_time_high = round(max_asset_value[ticker],2)
#     date = max_asset_date[ticker]

#     body_2 = f'     t/o {ticker}: {latest}€ ({change_prior_week}% to prior week)\n\
#        - Gain/Loss of {performance}€ ({change_to_purchase}%) to purchase value, which was {purchase}€\n\
#        - All-time high of {all_time_high}€ on {date}\n\n'

#     msg.attach(MIMEText(body_2))

# #For-loop does not work for attachment?!?
# part = MIMEBase('application', "octet-stream")
# with open(attachmentPath_1, 'rb') as file:
#     part.set_payload(file.read())
# encoders.encode_base64(part)
# part.add_header('Content-Disposition',
#                 'attachment; filename="{}"'.format(Path(attachmentPath_1).name))
# msg.attach(part)

# part = MIMEBase('application', "octet-stream")
# with open(attachmentPath_2, 'rb') as file:
#     part.set_payload(file.read())
# encoders.encode_base64(part)
# part.add_header('Content-Disposition',
#                 'attachment; filename="{}"'.format(Path(attachmentPath_2).name))
# msg.attach(part)

# part = MIMEBase('application', "octet-stream")
# with open(attachmentPath_3, 'rb') as file:
#     part.set_payload(file.read())
# encoders.encode_base64(part)
# part.add_header('Content-Disposition',
#                 'attachment; filename="{}"'.format(Path(attachmentPath_3).name))
# msg.attach(part)

# smtp = smtplib.SMTP(server, port)
# if use_tls:
#     smtp.starttls()
# smtp.login(username, password)
# smtp.sendmail(send_from, send_to, msg.as_string())
# smtp.quit()

# print ("Message Sent")